In [1]:
# Initialization cell
try:  # for CS1302 JupyterLite pyodide kernel
    import piplite

    with open("requirements.txt") as f:
        for package in f:
            package = package.strip()
            print("Installing", package)
            await piplite.install(package)
except ModuleNotFoundError:
    pass

# Generator

**CS1302 Introduction to Computer Programming**
___

In [2]:
%reload_ext divewidgets

## Recursion

Consider computing the [Fibonacci number](https://en.wikipedia.org/wiki/Fibonacci_number) of order $n$:

$$
F_n := 
\begin{cases}
F_{n-1}+F_{n-2} & n>1 \kern1em \text{(recurrence)}\\
1 & n=1 \kern1em \text{(base case)}\\
0 & n=0 \kern1em \text{(base case)}.
\end{cases}$$
Fibonacci numbers have practical applications in generating [pseudorandom numbers](https://en.wikipedia.org/wiki/Lagged_Fibonacci_generator).

**Can we define the function by calling the function itself?**

Try stepping through such a function below to see how it works:

In [3]:
%%optlite -r -h 450
def fibonacci(n):
    if n > 1:
        return fibonacci(n - 1) + fibonacci(n - 2)  # recursion
    elif n == 1:
        return 1
    else:
        return 0


print(fibonacci(2))

1


OPTWidget(value=None, height=450, script='def fibonacci(n):\n    if n > 1:\n        return fibonacci(n - 1) + …

```{important}

A function that calls itself (*recurs*) is called a [*recursion*](https://en.wikipedia.org/wiki/Recursion_(computer_science)).
```

**Exercise** 

Write a function `gcd` that implements the [Euclidean algorithm for the greatest common divisor](https://en.wikipedia.org/wiki/Euclidean_algorithm): 

$$\operatorname{gcd}(a,b)=\begin{cases}a & b=0\\ \operatorname{gcd}(b, a\operatorname{mod}b) & \text{otherwise.} \end{cases}$$

In [4]:
%%optlite -r -h 550
def gcd(a, b):
    return gcd(b, a % b) if b else a
    raise NotImplementedError()


print(gcd(3 * 5, 5 * 7)) # gcd = ?

5


OPTWidget(value=None, height=550, script='def gcd(a, b):\n    return gcd(b, a % b) if b else a\n    raise NotI…

**Is recursion strictly necessary?**

E.g., the following computes the Fibonacci number using a while loop instead.

In [5]:
%%optlite -r -h 550
def fibonacci_iteration(n):
    if n > 1:
        _, F = 0, 1  # next two Fibonacci numbers
        while n > 1:
            _, F, n = F, F + _, n - 1
        return F
    elif n == 1:
        return 1
    else:
        return 0


fibonacci_iteration(3)

2

OPTWidget(value=None, height=550, script='def fibonacci_iteration(n):\n    if n > 1:\n        _, F = 0, 1  # n…

In [6]:
# more tests
for n in range(5):
    assert fibonacci(n) == fibonacci_iteration(n)

**Exercise** 

We can always convert a recursion to an iteration. Why?

```{hint}
See the [recursion vs interaction](https://en.wikipedia.org/wiki/Recursion_(computer_science)#Recursion_versus_iteration).
```

**Answer**

Recursion always use a stack, which can be stored manually to implement an iteration.

**Exercise** 

Implement `gcd_iteration` using a while loop instead of recursion.

```{hint}

See [tail recursion](https://en.wikipedia.org/wiki/Recursion_(computer_science)#Tail-recursive_functions).
```

In [7]:
%%optlite -r -h 550
def gcd_iteration(a, b):
    while b:
        a, b = b, a % b
    return a


gcd_iteration(3 * 5, 5 * 7)

5

OPTWidget(value=None, height=550, script='def gcd_iteration(a, b):\n    while b:\n        a, b = b, a % b\n   …

In [8]:
# test
for n in range(5):
    assert fibonacci(n) == fibonacci_iteration(n)

**What are the benefits of recursion?**

- Recursion is often shorter and easier to understand.
- It can be easier to write code by *wishful thinking* or *[declarative programming](https://en.wikipedia.org/wiki/Declarative_programming)* as supposed to [imperative programming](https://en.wikipedia.org/wiki/Imperative_programming).

**Is recusion more efficient than iteration?**

**Exercise** 

Find the smallest values of `n` for `fibonacci(n)` and `fibonacci_iteration(n)` respectively to run for more than a second.

In [10]:
%%timeit -n 1
n = 10
fibonacci(n)

22 µs ± 6.5 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
%%timeit -n 1
n = 100000
fibonacci_iteration(n)

101 ms ± 9.95 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


To understand the difference in performance, modify `fibonacci` to print each function call as follows.

In [17]:
def fibonacci_verbose(n):
    """Returns the Fibonacci number of order n."""
    print(f"fibonacci({n})")
    return fibonacci_verbose(n - 1) + fibonacci_verbose(n - 2) if n > 1 else 1 if n == 1 else 0


fibonacci_verbose(5)

fibonacci(5)
fibonacci(4)
fibonacci(3)
fibonacci(2)
fibonacci(1)
fibonacci(0)
fibonacci(1)
fibonacci(2)
fibonacci(1)
fibonacci(0)
fibonacci(3)
fibonacci(2)
fibonacci(1)
fibonacci(0)
fibonacci(1)


5

There are many redundant computations. E.g., `fibonacci(3)` is called twice because

- `fibonacci(5)` calls `fibonacci(4)` and `fibonacci(3)`.
- `fibonacci(4)` then calls `fibonacci(3)` and `fibonacci(2)`.

## Global Variables and Closures

Consider generating a sequence of Fibonacci numbers:

In [18]:
for n in range(5):
    print(fibonacci_iteration(n))

0
1
1
2
3


**Exercise** 

Is the above loop efficient?

**Answer**

No. fibonacci_iteration(n) for smaller n value is calculated and can be used for larger n calculation.

**How to avoid redundant computations?**

One way is to store the last two computed Fibonacci numbers.

In [19]:
%%optlite -r -h 650
Fn, Fnn, n = 0, 1, 0  # global variables


def print_fibonacci_state():
    print(
        f"""Global states:
    Fn  : Next Fibonacci number      = {Fn}
    Fnn : Next next Fibonacci number = {Fnn}
    n   : Next order                 = {n}"""
    )


def next_fibonacci():
    """Returns the next Fibonacci number."""
    global Fn, Fnn, n  # global declaration
    value, Fn, Fnn, n = Fn, Fnn, Fn + Fnn, n + 1
    return value


for i in range(5):
    print(next_fibonacci())
print_fibonacci_state()

0
1
1
2
3
Global states:
    Fn  : Next Fibonacci number      = 5
    Fnn : Next next Fibonacci number = 8
    n   : Next order                 = 5


OPTWidget(value=None, height=650, script='Fn, Fnn, n = 0, 1, 0  # global variables\n\n\ndef print_fibonacci_st…

```{important}

Rules for [*global/local variables*](https://docs.python.org/3/faq/programming.html#what-are-the-rules-for-local-and-global-variables-in-python):
1. A local variable must be defined within a function.
1. An assignment defines a local variable except after a [`global` statement](https://docs.python.org/3/reference/simple_stmts.html#the-global-statement).
```

**Why `global` is NOT needed in `print_fibonacci_state`?**

Without ambiguity, `Fn, Fnn, n` in `print_fibonacci_state` are not local variables by Rule 1 because they are not defined within the function.

**Why `global` is needed in `next_fibonacci`?**

What would happen if the `global` statement is removed:

In [20]:
%%optlite -h 400
def next_fibonacci():
    """Returns the next Fibonacci number."""
    # global Fn, Fnn, n
    value = n
    n, Fnn, n = Fnn, n + Fnn, n + 1
    return value


next_fibonacci()

OPTWidget(value=None, height=400, script='def next_fibonacci():\n    """Returns the next Fibonacci number."""\…

`UnboundLocalError` is raised (as supposed to `NameError`) because

- the assignment in Line 5 defines `n` as a local variable by Rule 2, but
- the assignment in Line 4 references `n`, which is not yet defined at that point.

**Are global variables preferred over local ones?**

Consider rewriting the for loop as a while loop:

In [21]:
%%optlite -h 650
Fn, Fnn, n = 0, 1, 0  # global variables


def print_fibonacci_state():
    print(
        f"""Global states:
    Fn  : Next Fibonacci number      = {Fn}
    Fnn : Next next Fibonacci number = {Fnn}
    n   : Next order                 = {n}"""
    )


def next_fibonacci():
    """Returns the next Fibonacci number."""
    global Fn, Fnn, n  # global declaration
    value, Fn, Fnn, n = Fn, Fnn, Fn + Fnn, n + 1
    return value


n = 0
while n < 5:
    print(next_fibonacci())
    n += 1
print_fibonacci_state()

OPTWidget(value=None, height=650, script='Fn, Fnn, n = 0, 1, 0  # global variables\n\n\ndef print_fibonacci_st…

**Exercise** Why does the while loop print only 3 numbers instead of 5 Fibonacci numbers?

**Answer**

The while loop only execute `next_fibonacci()` for `n=0,2,4`.

To avoid such error, a convention in python is to use a leading underscore for variable names that are [*private*](https://www.python.org/dev/peps/pep-0008) (for internal use):  
> _single_leading_underscore: weak "internal use" indicator. E.g., from M import * does not import objects whose names start with an underscore.

In [ ]:
%%optlite -h 600
_Fn, _Fnn, _n = 0, 1, 0  # global variables


def print_fibonacci_state():
    print(
        f"""Global states:
    _Fn  : Next Fibonacci number      = {_Fn}
    _Fnn : Next next Fibonacci number = {_Fnn}
    _n   : Next order                 = {_n}"""
    )


def next_fibonacci():
    """Returns the next Fibonacci number."""
    global _Fn, _Fnn, _n  # global declaration
    value, _Fn, _Fnn, _n = _Fn, _Fnn, _Fn + _Fnn, _n + 1
    return value


n = 0
while n < 5:
    print(next_fibonacci())
    n += 1
print_fibonacci_state()

```{important}

Using global variables,
- codes are less predictable, more difficult to reuse/extend, and
- tests cannot be isolated, making debugging difficult.
```

**Is it possible to store the function states without using global variables?**

We can use nested functions and [`nonlocal` variables](https://docs.python.org/3/reference/simple_stmts.html#grammar-token-nonlocal-stmt).

In [12]:
def fibonacci_sequence(Fn, Fnn):
    def next_fibonacci():
        """Returns the next (generalized) Fibonacci number starting with
        Fn and Fnn as the first two numbers."""
        nonlocal Fn, Fnn, n  # declare nonlocal variables
        value = Fn
        Fn, Fnn, n = Fnn, Fn + Fnn, n + 1
        return value

    def print_fibonacci_state():
        print(
            """States:
        Next Fibonacci number      = {}
        Next next Fibonacci number = {}
        Next order                 = {}""".format(
                Fn, Fnn, n
            )
        )

    n = 0  # Fn and Fnn specified in the function arguments
    return next_fibonacci, print_fibonacci_state


next_fibonacci, print_fibonacci_state = fibonacci_sequence(0, 1)
n = 0
while n < 5:
    print(next_fibonacci())
    n += 1
print_fibonacci_state()

0
1
1
2
3
States:
        Next Fibonacci number      = 5
        Next next Fibonacci number = 8
        Next order                 = 5


The state variables `Fn, Fnn, n` are now [*encapsulated*](https://en.wikipedia.org/wiki/Encapsulation_(computer_programming)), and the functions returned by `fibonacci_sequence` no longer depend on any global variables.

Another benefit of using nested functions is creating different Fibonacci sequences with different base cases.

In [24]:
my_next_fibonacci, my_print_fibonacci_state = fibonacci_sequence("cs", "1302")
for n in range(5):
    print(my_next_fibonacci())
my_print_fibonacci_state()

cs
1302
cs1302
1302cs1302
cs13021302cs1302
States:
        Next Fibonacci number      = 1302cs1302cs13021302cs1302
        Next next Fibonacci number = cs13021302cs13021302cs1302cs13021302cs1302
        Next order                 = 5


`next_fibonacci` and `print_fibonacci_state` are *local functions* of `fibonacci_sequence`.  

```{important}
- Local functions can access (*capture*) the other local variables of `fibonacci_sequence` by forming the so-called *closures*.
- Similar to the `global` statement, a [`nonlocal` statement](https://docs.python.org/3/reference/simple_stmts.html#the-nonlocal-statement) is needed for assigning non-local variables.
```

Each local function has an attribute named `__closure__` that stores the captured local variables.

In [13]:
def print_closure(f):
    """Print the closure of a function."""
    print("closure of ", f.__name__)
    for cell in f.__closure__:
        print("    {} content: {!r}".format(cell, cell.cell_contents))


print_closure(next_fibonacci)
print_closure(print_fibonacci_state)

closure of  next_fibonacci
    <cell at 0x7f26b86eb250: int object at 0x7f26bdd0c170> content: 5
    <cell at 0x7f26936febf0: int object at 0x7f26bdd0c1d0> content: 8
    <cell at 0x7f26936fd900: int object at 0x7f26bdd0c170> content: 5
closure of  print_fibonacci_state
    <cell at 0x7f26b86eb250: int object at 0x7f26bdd0c170> content: 5
    <cell at 0x7f26936febf0: int object at 0x7f26bdd0c1d0> content: 8
    <cell at 0x7f26936fd900: int object at 0x7f26bdd0c170> content: 5


## Generator

An easier way to generate a sequence of objects one by one is to write a *generator*.

In [14]:
fibonacci_generator = (fibonacci_iteration(n) for n in range(3))
fibonacci_generator

<generator object <genexpr> at 0x7f2693574120>

The above uses a [*generator expression*](https://docs.python.org/3/reference/expressions.html#grammar-token-generator-expression) to define `fibonacci_generator`.

**How to obtain items from a generator?**

We can use the [`next` function](https://docs.python.org/3/library/functions.html#next).

In [15]:
fibonacci_generator = (fibonacci_iteration(n) for n in range(3))

while True:
    print(next(fibonacci_generator))  # raises StopIterationException eventually

0
1
1


StopIteration: 

A generator object is [*iterable*](https://www.programiz.com/python-programming/iterator), i.e., it implements both `__iter__` and `__next__` methods that are automatically called in a `for` loop as well as the `next` function.

In [16]:
fibonacci_generator = (fibonacci_iteration(n) for n in range(5))

for fib in fibonacci_generator:  # StopIterationException handled by for loop
    print(fib)

0
1
1
2
3


**Is `fibonacci_generator` efficient?**

No, again, due to redundant computations. A better way to define the generator is to use the keyword [`yield`](https://docs.python.org/3/reference/expressions.html?highlight=yield#yield-expressions):

In [17]:
%%optlite -h 450
def fibonacci_sequence(Fn, Fnn, stop):
    """Return a generator that generates Fibonacci numbers
    starting from Fn and Fnn until stop (exclusive)."""
    while Fn < stop:
        yield Fn  # return Fn and pause execution
        Fn, Fnn = Fnn, Fnn + Fn


for fib in fibonacci_sequence(0, 1, 5):
    print(fib)

OPTWidget(value=None, height=450, script='def fibonacci_sequence(Fn, Fnn, stop):\n    """Return a generator th…

```{important}

1. `yield` causes the function to return a *generator* without executing the function body.
1. Calling `__next__` resumes the execution, which 
    - pauses at the subsequent `yield` expression, if any, or
    - raises the `StopIterationException` at the end otherwise.
```

**Exercise** 

`yield` can be both a statement and an expression. As an expression: 
- The value of a `yield` expression is `None` by default, but 
- it can be set by the `generator.send` method.

Add the document string to the following function. In particular, explain the effect of calling the method `send` on the returned generator.

In [3]:
def fibonacci_sequence(Fn, Fnn, stop):
    while Fn < stop:
        print(f":{Fn}, {Fnn}")
        value = yield Fn
        if value is not None:
            Fnn = value  # set next number to the value of yield expression
        Fn, Fnn = Fnn, Fnn + Fn


fibonacci_generator = fibonacci_sequence(0, 1, 15)
print(next(fibonacci_generator))
print()
print(fibonacci_generator.send(2))
for fib in fibonacci_generator:
    print(fib)

:0, 1
0

:2, 2
2
:2, 4
2
:4, 6
4
:6, 10
6
:10, 16
10
